<a href="https://colab.research.google.com/github/kampelmuehler/MLKurs/blob/main/MNIST/MNIST_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup
# adapted from https://gist.github.com/rickkk856/6a2800cc84dd8fd456074e5a467edc47 and https://daily-dev-tips.com/posts/javascript-mouse-drawing-on-the-canvas/
! pip install onnxruntime --quiet
! wget -nc 'https://github.com/kampelmuehler/MLKurs/raw/main/MNIST/MNIST.onnx'
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
from IPython.display import display, HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import onnxruntime as ort

canvas_size = 256
img = None

js_code = '''
<style>
  div {
      display: flex;
  }
  canvas{border:1px solid white !important;cursor:crosshair}
</style>
<canvas id="myCanvas" width="%d" height="%d"></canvas>
<script>
  var canvas = document.querySelector('canvas')
  var ctx = canvas.getContext('2d')
  ctx.fillStyle = 'black';
  ctx.fillRect( 0, 0, canvas.width, canvas.height)
  var button = document.querySelector('button')
  let coord = { x: 0, y: 0 };
  canvas.onmousedown = ()=>{
    canvas.addEventListener('mousemove', draw)
    reposition(event);
  }
  canvas.onmouseup = ()=>{
    canvas.removeEventListener('mousemove', draw)
  }
  function reposition(event) {
    coord.x = event.clientX - canvas.offsetLeft;
    coord.y = event.clientY - canvas.offsetTop;
  }

  function draw (event) {
    ctx.beginPath();
    ctx.lineWidth = 27;
    ctx.lineCap = "round";
    ctx.strokeStyle = "white";
    ctx.moveTo(coord.x, coord.y);
    reposition(event);
    ctx.lineTo(coord.x, coord.y);
    ctx.stroke();
  }
    // FINISH BUTTON
  var data = new Promise(resolve=>{
    button.onclick = ()=>{
      resolve(canvas.toDataURL('image/jpg'))
    }
  })
</script>
'''


## main drawing function
def draw(w=canvas_size, h=canvas_size):
  display(HTML(js_code % (w, h)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])  

# Action for Save Button
def on_button_clicked(b):
  global img
  with output:
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    nparr = np.frombuffer(binary, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    x = (img[None, None, :, :].astype(np.float32) / 255 - 0.1307) / 0.3081
    ort_sess = ort.InferenceSession('MNIST.onnx')
    outputs = ort_sess.run(None, {'input': x})

    # Print Result 
    print(f"You have drawn a {np.argmax(outputs[0])}")

In [ ]:
## Show Save Button & Save outputs
button = widgets.Button(description="Submit")
button.on_click(on_button_clicked)
output = widgets.Output()
display(button, output)

## Show Canvas for the First Time
draw()